# IMDb Database



In [ ]:
import sqlite3

conn = sqlite3.connect('IMDb.db')

print ("Opened database successfully")

In [1]:
# import library 
import pandas as pd
import sqlite3
import random
import uuid

In [ ]:
# function to clean the country column 
def change_country (df, name): # df <- name of the df & name <- name of the column
    for i in range(len(df[name])):
        if str(df[name][i]) == "nan":
            df[name][i] = "Not Available"
        else:
            country_array = df[name][i].split(",")
        
            if isinstance(country_array, list):
                df[name][i] = country_array[0].strip()
                
    return df
    

## Movie Table 

In [ ]:
# importing the movie data 
movie_df = pd.read_csv('../database_data/movies.csv')
# changing country format
movie_df = change_country(movie_df, "country")
# dropping columns
movie_df = movie_df.drop(columns=['date_added', 'type', 'listed_in'])
# renaming
movie_df = movie_df.rename(columns = {"rating": "is_adult"})

In [ ]:
# changing duration into float 
duration_array = []

for i in range(len(movie_df)):
    number = str(movie_df["duration"][i]).split()

    if number[0] == "nan":
        duration_array.append(60)
    else:
        duration_array.append(int(number[0]))
    
movie_df["duration"] = duration_array
movie_df

## Genre Table

In [ ]:
# improting the original dataset to get all the genre 
full_df = pd.read_csv('../original data/netflix_titles.csv')

In [ ]:
full_df = pd.DataFrame('../Data')

In [ ]:
# normalizing the genre and creating each genre to have genre id 
genre = {}

for i in range(len(full_df)):
    
    genres = full_df["listed_in"][i].split(",")
    
    for j in genres:
        j = j.strip()
        if j not in genre.keys():
            genre[j] = uuid.uuid4()

genre_df = pd.DataFrame()
genre_df["genre_type"] = genre.keys() 
genre_df["genre_id"] = genre.values()
genre_df

## TV Shows Table

In [ ]:
shows_df = pd.read_csv("../database_data/tv_shows.csv")
# changing country format
shows_df = change_country(shows_df, "country")
# dropping columns
shows_df = shows_df.drop(columns=['date_added', 'type', 'listed_in'])
# renaming
shows_df = shows_df.rename(columns = {"rating": "is_adult", "duration": "num_seasons"})

In [ ]:
season_array = []

for i in range(len(shows_df)):
    number = str(shows_df["num_seasons"][i]).split()

    if number[0] == "nan":
        season_array.append(60)
    else:
        season_array.append(int(number[0]))
    
shows_df["num_seasons"] = season_array
shows_df

## Movie/Show Table
This is a table that contain each movie and their genres.

In [ ]:
# matching each movie or series to their respective genre
movie_show_genre = {}

# looping every row and matching the correct genre Id
for i in range(len(full_df)):
    show_id = full_df["show_id"][i]
    genres = full_df["listed_in"][i].split(',')
    
    if len(genres) == 1:
        genre_id = genre[genres[0]] 
        movie_show_genre[show_id] = genre_id
    else:
        for j in genres:
            j = j.strip()
            genre_id = genre[j]
            if show_id in movie_show_genre: 
                contains = movie_show_genre[show_id]
                movie_show_genre[show_id] = [genre_id , *contains]
            else:
                movie_show_genre[show_id] = [genre_id]

In [ ]:
movie_show_df = pd.DataFrame()

movie_show_id_column = []

for i in movie_show_genre.keys():
    id_array = movie_show_genre[i]
    if isinstance(id_array, list):
        for j in id_array:
            movie_show_id_column.append(i)
    else:
        movie_show_id_column.append(i)

def get_all_values(d):
    genre_id_array = []
    for i in d.keys():
        if isinstance(d[i], list):
            for j in d[i]:
                genre_id_array.append(j)
        else:
                genre_id_array.append(d[i])
                
    return genre_id_array
            
        
movie_show_df["movie_show_id"] = movie_show_id_column
movie_show_df["genre_id"] = get_all_values(movie_show_genre)
movie_show_df

## Comment table based on Review 

In [2]:
# imorting the review csv 
review_df = pd.read_csv('../database_data/populate_data/comments.csv')
# changing comments_id name to review_id
review_df = review_df.rename(columns = {"comments_id": "review_id"})
review_df

,review_id,show_id,user_id,reviews,ratings
0,1,s6113,3851,One of the other reviewers has mentioned that ...,9.35
1,2,s5170,5370,A wonderful little production. <br /><br />The...,7.94
2,3,s5812,4012,I thought this was a wonderful way to spend ti...,8.91
3,4,s4430,5412,Basically there's a family where a little boy ...,0.33
4,5,s8213,1570,"Petter Mattei's ""Love in the Time of Money"" is...",6.82
...,...,...,...,...,...
49995,49996,s661,3371,I thought this movie did a down right good job...,6.24
49996,49997,s1489,1000,"Bad plot, bad dialogue, bad acting, idiotic di...",1.53
49997,49998,s6874,5922,I am a Catholic taught in parochial elementary...,2.52
49998,49999,s94,1412,I'm going to have to disagree with the previou...,1.48


In [3]:
# importing the new comment 
random_df = pd.read_csv('/Users/hengding/Desktop/train-balanced-sarcasm.csv')
random_df = random_df.drop(range(200001,1010824))
random_df = random_df.reset_index(drop=True)
random_df

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...
...,...,...,...,...,...,...,...,...,...,...
199998,0,Its in your hands,InsertPotatoes,pics,1,1,0,2016-07,2016-07-14 05:24:20,can you see the cell phone? :)
199999,0,Getter's remix of Slam Dunk or Rip N Dip Remixes?,Revo3,skrillex,2,2,0,2016-07,2016-07-28 13:43:16,Slam Dunk is on the interwebs along with the G...
200000,0,I had it...Until the wieners tried to make the...,AFrogsLife,Dachshund,2,2,0,2016-06,2016-06-29 20:28:01,Every Dachshund Lover Should Own This
200001,1,The Slavs got their own country - it is called...,catsi,worldnews,1,1,0,2009-01,2009-01-23 21:12:49,I've always been unsettled by that. I hear a l...


In [16]:
# creating comment id 
comment_id_array = []
comment_id_array_parent = []
comment_id_array_child = []

for i in range(len(random_df) * 2):
    comment_id_1 = f"C-{i}"
    comment_id_2 = f"C-Reply-{i}"
    random_array = [comment_id_1, comment_id_2]

    random_num = random.randint(0,1)
    
    comment_id_array.append(random_array[random_num])
    
    if random_num == 0:
        comment_id_array_parent.append(random_array[random_num])
    else:
        comment_id_array_child.append(random_array[random_num])
    

In [17]:
# creating all comment 
comment_array = []

for i in range(len(random_df)):
    comment_array.append(random_df["comment"][i])
    comment_array.append(random_df["parent_comment"][i])

In [18]:
# creating comment data frame 
comment_df = pd.DataFrame()

comment_df["comment_id"] = comment_id_array
comment_df["comment"] = comment_array

comment_df

,comment_id,comment
0,C-0,NC and NH.
1,C-Reply-1,"Yeah, I get that argument. At this point, I'd ..."
2,C-2,You do know west teams play against west teams...
3,C-Reply-3,The blazers and Mavericks (The wests 5 and 6 s...
4,C-Reply-4,"They were underdogs earlier today, but since G..."
...,...,...
400001,C-Reply-400001,Every Dachshund Lover Should Own This
400002,C-400002,The Slavs got their own country - it is called...
400003,C-400003,I've always been unsettled by that. I hear a l...
400004,C-400004,"values, as in capitalism .. there is good mone..."


In [44]:
# export into csv 
# this is the comment_info 
comment_df.to_csv("all_comment_info.csv", index=False)

In [19]:
print(len(comment_id_array_parent), len(comment_id_array_child))

199949 200057


In [21]:
user_df = pd.read_csv("../database_data/populate_data/users.csv")
user_df

,user_id,email_address,name
0,1,Shelby_Dillon3211@elnee.tech,Shelby Dillon
1,2,Taylor_Tait5975@gompie.com,Taylor Tait
2,3,Eve_Radcliffe7510@ovock.tech,Eve Radcliffe
3,4,Wade_Nobbs8318@cispeto.com,Wade Nobbs
4,5,Enoch_Jennson5292@grannar.com,Enoch Jennson
...,...,...,...
8995,8996,Mabel_Thorne4997@bauros.biz,Mabel Thorne
8996,8997,Hannah_Dixon7128@iatim.tech,Hannah Dixon
8997,8998,Nina_Neville425@sheye.org,Nina Neville
8998,8999,Chad_Butler1528@qater.org,Chad Butler


In [27]:
# this part will be creating 
# parent_comment_id
# child_comment_id
# review id 
# user id 

# replicating more parents so the length of the parents and child comment is the same 

for i in range(len(comment_id_array_child) - len(comment_id_array_parent)):
    
    random_generator = int(random.random() * len(comment_id_array_parent))
    
    random_parent_id = comment_id_array_parent[random_generator]
    
    comment_id_array_parent.append(random_parent_id)
    

In [36]:
# creating more reviews to match the lenght of the comment_2_df

review_array = list(review_df["review_id"].values)

for i in range(len(comment_id_array_parent) - len(review_df)):
    
    random_generator = int(random.random() * len(review_array))
    
    new_random_review = review_array[random_generator]
    
    review_array.append(new_random_review)
    
    

In [40]:
# creating more user count to match the lenght of the comment_2_df

user_array = list(user_df["user_id"].values)
    
for i in range(len(comment_id_array_parent) - len(user_df)):
    
    random_generator = int(random.random() * len(user_array))
    
    new_random_user = user_array[random_generator]
    
    user_array.append(new_random_user)


In [37]:
len(review_array)

200057

In [42]:
# splitting parent and child comment 
comment_2_df = pd.DataFrame()

comment_2_df["parent_comment_id"] = comment_id_array_parent
comment_2_df["child_comment_id"] = comment_id_array_child
comment_2_df["review_id"] = review_array
comment_2_df["user_id"] = user_array

comment_2_df

,parent_comment_id,child_comment_id,review_id,user_id
0,C-0,C-Reply-1,1,1
1,C-2,C-Reply-3,2,2
2,C-5,C-Reply-4,3,3
3,C-6,C-Reply-7,4,4
4,C-8,C-Reply-9,5,5
...,...,...,...,...
200052,C-202301,C-Reply-399993,46007,673
200053,C-341358,C-Reply-399997,21468,1252
200054,C-17451,C-Reply-399999,37596,975
200055,C-51475,C-Reply-400001,802,6562


In [45]:
comment_2_df.to_csv("comment_reply.csv", index=False)

In [48]:
# creating random up vote and down vote 
up_vote = []
down_vote = []

for i in range(len(review_df)):
    up = random.randint(0,9000)
    down = random.randint(0,400)
    
    up_vote.append(up)
    down_vote.append(down)
    

In [49]:
# up vote df
review_up_vote_df = pd.DataFrame()
review_up_vote_df["review_id"] = list(review_df["review_id"].values)
review_up_vote_df["up_vote"] = up_vote

review_up_vote_df


,review_id,up_vote
0,1,1016
1,2,1762
2,3,4484
3,4,1972
4,5,5884
...,...,...
49995,49996,866
49996,49997,3281
49997,49998,7231
49998,49999,1082


In [50]:
# down vote df
review_down_vote_df = pd.DataFrame()
review_down_vote_df["review_id"] = list(review_df["review_id"].values)
review_down_vote_df["down_vote"] = down_vote

review_down_vote_df


,review_id,down_vote
0,1,322
1,2,345
2,3,69
3,4,122
4,5,163
...,...,...
49995,49996,322
49996,49997,220
49997,49998,358
49998,49999,376


In [51]:
# covert down vote and up vote into csv 
review_up_vote_df.to_csv("up_vote.csv", index=False)
review_down_vote_df.to_csv("down_vote.csv", index=False)